In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

# ignore warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore')

In [6]:
sentiment_analysis_df = pd.read_csv("./main_df_output/sentiment_analysis_df.csv")
sentiment_analysis_df_with_price = pd.read_csv('./main_df_output/sentiment_analysis_df_with_price_2.csv')

In [24]:
sentiment_analysis_df_with_price.head(5)

,company_name,Group,ticker,file_name,date,WA_sentiment,Close,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.017045,1655.0,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.008005,1353.0,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.014271,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.000225,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.030718,1144.0,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860


In [28]:
year = sentiment_analysis_df_with_price['date'].apply(lambda r: r[:4])
years = [int(i) for i in year]
len(years)

1817

In [8]:
df = sentiment_analysis_df_with_price.copy()
ctg_list = df['Group'].unique().tolist()
day_list = sentiment_analysis_df_with_price.columns[7:]
column_day = day_list

def calculate_percentage(ctg,day):
    df_group = df[df['Group']==ctg]
    def check_trend(sentiment_list, change_prc_list):
        tag_list = []
        for sentiment,change_prc in zip(sentiment_list,change_prc_list):
            if ((sentiment >0)&(change_prc>0)):
                tag = 'upward'
            elif ((sentiment <0)&(change_prc<0)):
                tag = 'downward'
            else:
                tag = 'No'
            tag_list.append(tag)
        return tag_list

    sentiment_list = df_group['WA_sentiment']
    change_prc_list = df_group[day]
    trend_list = Counter(check_trend(sentiment_list, change_prc_list))
    total = trend_list['upward'] + trend_list['downward'] + trend_list['No'] 
    correct_predict = (trend_list['upward'] + trend_list['downward'])/total

    return (correct_predict)

# ctg = 'Japanese & Pacific'
# day = 'D-1'

# calculate_percentage(ctg,day)

# empty df
df_result_1 = pd.DataFrame(np.zeros((10,13)))
col_name = ['category']
col_name.extend(day_list)
df_result_1.columns = col_name
df_result_1['category'] = ctg_list
df_result_1 = df_result_1.set_index(['category'])

# calculate
for day in day_list:
    for ctg in ctg_list:
        df_result_1.at[ctg,day] = calculate_percentage(ctg,day)
# df_result_1

# plot
import matplotlib as mpl
def style(v, props=''):
    return props if v >= 0.5 else None
s2 = df_result_1.style.applymap(style, props='color:orange;')\
              #.applymap(lambda v: 'opacity: 20%;' if (v < 0.5) and (v > -0.3) else None)
s2


,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
category,,,,,,,,,,,,
Motor/Personal,0.568182,0.409091,0.575758,0.431818,0.545455,0.522727,0.568182,0.507576,0.583333,0.439394,0.606061,0.431818
Global Commercial,0.521505,0.500000,0.505376,0.569892,0.510753,0.526882,0.494624,0.489247,0.505376,0.500000,0.548387,0.505376
London Market,0.506849,0.479452,0.547945,0.452055,0.561644,0.438356,0.575342,0.397260,0.561644,0.465753,0.575342,0.328767
LN Equity,0.750000,0.250000,0.750000,0.250000,0.500000,0.375000,0.625000,0.250000,0.375000,0.125000,0.375000,0.375000
US - Specialty/P&C/Reinsurance,0.522248,0.540984,0.533958,0.550351,0.550351,0.562061,0.578454,0.590164,0.580796,0.576112,0.592506,0.545667
European (Re)Insurers,0.535484,0.541935,0.470968,0.503226,0.509677,0.509677,0.574194,0.548387,0.612903,0.567742,0.612903,0.587097
Japanese & Pacific,0.558333,0.541667,0.550000,0.508333,0.508333,0.550000,0.508333,0.525000,0.550000,0.550000,0.583333,0.525000
Run-off,0.517857,0.642857,0.517857,0.571429,0.500000,0.642857,0.517857,0.589286,0.589286,0.625000,0.589286,0.642857
Life Groups & Retail Life,0.549296,0.514085,0.566901,0.496479,0.538732,0.492958,0.521127,0.478873,0.549296,0.482394,0.577465,0.440141


In [15]:
from scipy.stats import pearsonr
data1 = df['WA_sentiment']
for d in day_list:
    data2 = df[d]
    # calculate Pearson's correlation
    corr, _ = pearsonr(data1, data2)
    print(d)
    print('Pearsons correlation: %.3f' % corr)
    print('-----------')

D-1
Pearsons correlation: 0.037
-----------
D+1
Pearsons correlation: -0.034
-----------
D-2
Pearsons correlation: 0.037
-----------
D+2
Pearsons correlation: 0.003
-----------
D-3
Pearsons correlation: 0.028
-----------
D+3
Pearsons correlation: 0.024
-----------
D-5
Pearsons correlation: 0.040
-----------
D+5
Pearsons correlation: 0.005
-----------
D-10
Pearsons correlation: 0.014
-----------
D+10
Pearsons correlation: -0.002
-----------
D-15
Pearsons correlation: 0.043
-----------
D+15
Pearsons correlation: -0.019
-----------


In [22]:
# regression
from pygam.datasets import wage
from pygam import LinearGAM, s, f, PoissonGAM, GammaGAM, GAM

from sklearn.preprocessing import OneHotEncoder

import plotly.express as px


# label encoder
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

df_run = pd.DataFrame()
df_run['group'] = labelencoder.fit_transform(df['Group'])
df_run['sentiment'] = df['WA_sentiment']
df_run = df_run.join(df[column_day])
df_run

,group,sentiment,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,6,0.017045,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,6,0.008005,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,6,0.014271,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,6,0.000225,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,6,0.030718,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812,7,0.009128,0.063332,-0.018274,0.069258,-0.019425,0.073609,-0.060778,0.063179,-0.080203,0.031990,0.000203,0.026255,-0.019628
1813,7,0.024994,0.034267,-0.013371,0.041432,-0.016994,0.039294,-0.032275,0.050876,-0.041101,0.061678,-0.054802,0.054013,-0.033263
1814,7,0.019457,-0.016541,-0.011460,-0.027909,-0.011521,-0.015945,-0.015218,-0.009157,-0.027663,-0.003744,0.002403,-0.002704,0.041341
1815,7,0.037752,0.001828,-0.002943,0.008669,0.000706,0.007592,-0.002826,0.000294,0.000530,0.001238,0.026725,-0.010427,0.036320


In [18]:
fig = px.scatter(df_run, x="sentiment", y="D-1", marginal_x="histogram", marginal_y="histogram")
fig.show()

In [19]:
import statsmodels.api as sm

# TAKING PART OF THE DATA: only same direction one 
df1 = df_run[((df_run['sentiment']<0) & (df_run['D-1']<0)) | ((df_run['sentiment']>0) & (df_run['D-1']>0))]

X = df1['sentiment']
X = sm.add_constant(X)
Y = df1['D-1']
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    D-1   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     52.79
Date:                Mon, 15 Aug 2022   Prob (F-statistic):           7.60e-13
Time:                        21:47:12   Log-Likelihood:                 2179.9
No. Observations:                 978   AIC:                            -4356.
Df Residuals:                     976   BIC:                            -4346.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0146      0.001     11.811      0.000       0.012       0.017
sentiment      0.3624      0.050      7.266      0.000       0.265       0.460
==============================================================================
Omnibus:                      234.906   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1622.230
Skew:                           0.912   Prob(JB):                         0.00
Kurtosis:                       9.040   Cond. No.                         59.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
# with only same direction data
X = df1[['sentiment','group']].values.tolist()
Y = df1['D-1']

gam = GAM(s(0)+f(1)).fit(X, Y)
gam.summary()

GAM                                                                                                       
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     20.5331
Link Function:                     IdentityLink Log Likelihood:                               -846109.6691
Number of Samples:                          978 AIC:                                          1692262.4044
                                                AICc:                                           1692263.42
                                                GCV:                                                0.0006
                                                Scale:                                              0.0006
                                                Pseudo R-Squared:                                   0.2313
Feature Function                  Lam

In [33]:
# with all data
X = df_run[['sentiment','group']].values.tolist()
Y = df_run['D-1']

gam = GAM(s(0)+f(1)).fit(X, Y)
gam.summary()

GAM                                                                                                       
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     19.0853
Link Function:                     IdentityLink Log Likelihood:                               -678471.3522
Number of Samples:                         1817 AIC:                                          1356982.8751
                                                AICc:                                         1356983.3468
                                                GCV:                                                0.0013
                                                Scale:                                              0.0013
                                                Pseudo R-Squared:                                   0.0166
Feature Function                  Lam

In [35]:
libor = pd.read_excel('/Users/hienanh/Desktop/KBRA/1_month.xlsx')
libor

,Date,PX_LAST
0,2022-08-05,1.7227
1,2022-08-04,1.6730
2,2022-08-03,1.6654
3,2022-08-02,1.6478
4,2022-08-01,1.6255
...,...,...
8256,1989-11-07,15.1250
8257,1989-11-06,15.1875
8258,1989-11-03,15.2500
8259,1989-11-02,15.2500


In [39]:
conda install eikon

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import eikon as ek

In [45]:
libor = libor.rename(columns={'Date':'date'})
libor['date'] = libor['date'].astype(str)

In [50]:
#london = df[df.Group == 'London Market']
df_2 = df.merge(libor, how = 'inner', on = 'date')
df_2

,company_name,Group,ticker,file_name,date,WA_sentiment,Close,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15,PX_LAST
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.017045,1655.000000,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317,0.61250
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.008005,1353.000000,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825,0.65469
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.014271,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437,0.72306
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.000225,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437,0.72306
4,Ageas SA-NV-,Motor/Personal,AGS.BR,20111109_Ageas_SA-NV-_Earnings_Call_2011-11-9_...,2011-11-09,0.017093,13.500000,-0.054622,-0.028889,-0.031564,0.004444,-0.029475,0.011852,-0.010989,-0.044444,-0.055283,-0.197037,-0.031564,-0.037037,0.72306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,Helvetia Holding,Other insurers,HELN.SW,20200124_Helvetia_Holding_AG-_M-A_Call_2020-1-...,2020-01-24,0.021898,144.000000,0.022727,-0.020139,0.018388,-0.021528,0.026372,-0.016667,0.027837,-0.036111,0.045752,0.001389,0.042723,0.008333,0.63063
1805,Helvetia Holding,Other insurers,HELN.SW,20210325_Helvetia_Holding_AG-_Earnings_Call_20...,2021-03-25,0.020510,110.099998,0.004562,0.006358,0.011948,0.015441,0.011948,0.026340,0.014747,0.016349,0.020389,0.020890,0.036723,0.009083,0.04950
1806,Helvetia Holding,Other insurers,HELN.SW,20210622_Helvetia_Holding_AG-_Shareholder_Mtg_...,2021-06-22,0.011296,102.199997,0.000000,-0.002935,0.013889,0.003914,-0.010649,0.010763,0.016915,-0.017612,0.028686,-0.024462,0.010880,-0.018591,0.05413
1807,INTACT FINANCIAL-,Other insurers,IFC.TO,20120208_INTACT_FINANCIAL-_Earnings_Call_2012-...,2012-02-08,0.013560,60.400002,-0.007395,0.005298,0.014615,0.001656,0.013423,0.004967,0.019754,-0.006623,0.040124,-0.008444,0.041200,-0.016556,0.76256


In [51]:
df_2['group'] = labelencoder.fit_transform(df_2['Group'])

In [53]:
# with all data
X = df_2[['WA_sentiment','PX_LAST', 'group']].values.tolist()
Y = df_2['D-1']

gam = LinearGAM(s(0)+s(1)+f(2)).fit(X, Y)
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     31.1606
Link Function:                     IdentityLink Log Likelihood:                               -671157.7299
Number of Samples:                         1809 AIC:                                           1342379.781
                                                AICc:                                         1342380.9821
                                                GCV:                                                0.0013
                                                Scale:                                              0.0013
                                                Pseudo R-Squared:                                   0.0266
Feature Function                  Lam

In [54]:
# TAKING PART OF THE DATA: only same direction one 
df_same = df_2[((df_2['WA_sentiment']<0) & (df_2['D-1']<0)) | ((df_2['WA_sentiment']>0) & (df_2['D-1']>0))]

# with all data
X = df_same[['WA_sentiment','PX_LAST', 'group']].values.tolist()
Y = df_same['D-1']

gam = LinearGAM(s(0)+s(1)+f(2)).fit(X, Y)
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     31.2373
Link Function:                     IdentityLink Log Likelihood:                               -844249.8875
Number of Samples:                          973 AIC:                                          1688564.2496
                                                AICc:                                           1688566.53
                                                GCV:                                                0.0006
                                                Scale:                                              0.0006
                                                Pseudo R-Squared:                                   0.2519
Feature Function                  Lam

In [56]:
df_location = ['LSE','LSE','LSE','LSE','Brussels','XETRA','Paris',
                    'Swiss','Milan','LSE','LSE','LSE','LSE','NYSE',
                    'NYSE','NYSE','NYSE','NYSE','NYSE','NYSE','NYSE','NYSE','NYSE',
                    'NYSE','Paris','XETRA','Swiss','XETRA','Japan',
                    'Japan','Japan','ASX','NYSE','Toronto','LSE','LSE',
                    'LSE','LSE','LSE','LSE','LSE','LSE','Amsterdam',
                    'Johannesburg','Johannesburg','Oslo','Oslo','Milan','Amsterdam',
                    'Copenhagen','Swiss','Helsinki','Copenhagen','IOB','Swiss',
                    'Toronto'
    ]
ticker = sentiment_analysis_df_with_price['ticker'].unique()
df_country = pd.DataFrame({'Stock_market':df_location,'ticker':ticker})
# df_country

df_merge = df_2.merge(df_country, how='inner', on='ticker')
df_merge

,company_name,Group,ticker,file_name,date,WA_sentiment,Close,D-1,D+1,D-2,...,D+3,D-5,D+5,D-10,D+10,D-15,D+15,PX_LAST,group,Stock_market
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.017045,1655.000000,-0.031030,0.013897,-0.020130,...,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317,0.61250,6,LSE
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.008005,1353.000000,-0.118567,-0.054693,-0.110454,...,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825,0.65469,6,LSE
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.014271,887.500000,-0.256077,-0.076056,-0.247881,...,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437,0.72306,6,LSE
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.000225,887.500000,-0.256077,-0.076056,-0.247881,...,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437,0.72306,6,LSE
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.030718,1144.000000,0.100000,0.015734,0.095785,...,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860,0.72100,6,LSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,MS-AD INSURANCE-,Japanese & Pacific,8725.T,20190910_MS-AD_INSURANCE-_Guidance_Call_2019-9...,2019-09-10,0.001359,3464.000000,0.007563,0.024249,0.012865,...,0.049076,0.025155,0.032621,0.038681,0.057159,-0.004598,-0.010393,0.70975,2,Japan
1805,MS-AD INSURANCE-,Japanese & Pacific,8725.T,20191127_MS-AD_INSURANCE-_Guidance_Call_2019-1...,2019-11-27,0.005773,3573.000000,-0.001955,-0.004758,0.010464,...,-0.003638,0.016501,-0.005038,0.021441,0.023790,0.004781,0.037783,0.71338,2,Japan
1806,Enstar Group,Run-off,ESGR,20160615_Enstar_Group_Ltd-_Shareholder_Mtg_Cal...,2016-06-15,-0.008121,154.419998,-0.011459,0.008224,-0.011522,...,0.003108,-0.007073,0.006800,-0.014361,0.021111,-0.008348,0.032250,0.50631,8,NYSE
1807,Enstar Group,Run-off,ESGR,20170629_Enstar_Group_Ltd-_Shareholder_Mtg_Cal...,2017-06-29,0.013247,200.850006,0.008283,-0.010954,0.031852,...,-0.012198,0.031587,0.009709,0.015933,0.005228,0.038790,0.022156,0.25575,8,NYSE


In [57]:
# TAKING PART OF THE DATA: only same direction one 
df_cty = df_merge[((df_merge['WA_sentiment']<0) & (df_merge['D-1']<0)) | ((df_merge['WA_sentiment']>0) & (df_merge['D-1']>0))]
df_cty['region'] = labelencoder.fit_transform(df_cty['Stock_market'])

# with all data
X = df_cty[['WA_sentiment','PX_LAST', 'group','region']].values.tolist()
Y = df_cty['D-1']

gam = LinearGAM(s(0)+s(1)+f(2)+f(3)).fit(X, Y)
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     44.2703
Link Function:                     IdentityLink Log Likelihood:                               -851653.5938
Number of Samples:                          973 AIC:                                          1703397.7283
                                                AICc:                                         1703402.2489
                                                GCV:                                                0.0006
                                                Scale:                                              0.0005
                                                Pseudo R-Squared:                                   0.2788
Feature Function                  Lam